In [7]:
import zipfile
import os


with zipfile.ZipFile("/content/GPUPy.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [8]:
with zipfile.ZipFile("/content/GPUPy.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:

import numpy as np
import cupy as cp
import time
import matplotlib.pyplot as plt

def benchmark_matrix_multiplication():
    print("=== Matrix Multiplication Benchmark (CPU vs GPU) ===")
    print("This benchmark demonstrates GPU advantage in highly parallelizable tasks.\n")

    # Matrix dimensions to test (N x N)
    matrix_sizes = [100, 500, 1000, 2000, 4000, 8000]

    cpu_times = []
    gpu_times = []
    speedups = []

    # GPU Warm-up
    print("Warming up GPU...")
    try:
        a_gpu_warmup = cp.random.rand(10, 10)
        b_gpu_warmup = cp.random.rand(10, 10)
        _ = a_gpu_warmup @ b_gpu_warmup # Perform a dummy operation
        cp.cuda.Stream.null.synchronize() # Wait for GPU operations to complete
        print("GPU ready.")
    except Exception as e:
        print(f"GPU warm-up failed: {e}. Ensure CuPy is installed and GPU is available.")
        print("Skipping GPU benchmarks.")
        run_gpu = False
    else:
        run_gpu = True
    print("-" * 50)

    for n in matrix_sizes:
        print(f"\nBenchmarking {n}x{n} Matrix Multiplication:")

        # Generate random matrices on CPU
        a_cpu = np.random.rand(n, n)
        b_cpu = np.random.rand(n, n)

        # CPU Benchmark
        start_cpu = time.time()
        c_cpu = a_cpu @ b_cpu # Matrix multiplication using NumPy
        cpu_time = time.time() - start_cpu
        cpu_times.append(cpu_time)
        print(f"CPU Time ({n}x{n}): {cpu_time:.4f}s")

        if run_gpu:
            # Transfer matrices to GPU
            a_gpu = cp.asarray(a_cpu)
            b_gpu = cp.asarray(b_cpu)

            # GPU Benchmark
            start_gpu = time.time()
            c_gpu = a_gpu @ b_gpu # Matrix multiplication using CuPy
            cp.cuda.Stream.null.synchronize() # Ensure GPU operations are complete
            gpu_time = time.time() - start_gpu
            gpu_times.append(gpu_time)

            speedup = cpu_time / gpu_time
            speedups.append(speedup)
            print(f"GPU Time ({n}x{n}): {gpu_time:.4f}s")
            print(f"Speedup ({n}x{n}): {speedup:.2f}x")

            # Verify results (optional, but good practice)
            c_gpu_np = cp.asnumpy(c_gpu)
            if not np.allclose(c_cpu, c_gpu_np, rtol=1e-5, atol=1e-8):
                print("WARNING: CPU and GPU results differ!")
        else:
            gpu_times.append(np.nan)
            speedups.append(np.nan)

        print("-" * 50)

    # Plotting Results
    plt.figure(figsize=(10, 6))
    plt.loglog(matrix_sizes, cpu_times, 'o-', label='CPU Time', color='blue')
    if run_gpu:
        plt.loglog(matrix_sizes, gpu_times, 's-', label='GPU Time', color='orange')

    plt.title('Matrix Multiplication Performance (CPU vs GPU)')
    plt.xlabel('Matrix Dimension (N)')
    plt.ylabel('Execution Time (s)')
    plt.legend()
    plt.grid(True, which="both", ls="-", alpha=0.2)

    # Add speedup annotations for GPU
    if run_gpu:
        for i, n in enumerate(matrix_sizes):
            if not np.isnan(gpu_times[i]):
                plt.annotate(f'{speedups[i]:.1f}x',
                             (n, gpu_times[i]),
                             textcoords="offset points", xytext=(0,10), ha='center',
                             bbox=dict(boxstyle="round,pad=0.3", fc="yellow", ec="b", lw=0.5, alpha=0.7))

    plt.tight_layout()
    plt.savefig('matrix_multiplication_benchmark_gpu_advantage.png')
    plt.show()

if __name__ == "__main__": # >>> Bu satırı düzelttim! <<<
    benchmark_matrix_multiplication()


=== Matrix Multiplication Benchmark (CPU vs GPU) ===
This benchmark demonstrates GPU advantage in highly parallelizable tasks.

Warming up GPU...
GPU ready.
--------------------------------------------------

Benchmarking 100x100 Matrix Multiplication:
CPU Time (100x100): 0.0002s
GPU Time (100x100): 0.0082s
Speedup (100x100): 0.03x
--------------------------------------------------

Benchmarking 500x500 Matrix Multiplication:
CPU Time (500x500): 0.0057s
GPU Time (500x500): 0.0323s
Speedup (500x500): 0.18x
--------------------------------------------------

Benchmarking 1000x1000 Matrix Multiplication:
CPU Time (1000x1000): 0.0466s
GPU Time (1000x1000): 0.0359s
Speedup (1000x1000): 1.30x
--------------------------------------------------

Benchmarking 2000x2000 Matrix Multiplication:
CPU Time (2000x2000): 0.2810s
GPU Time (2000x2000): 0.1699s
Speedup (2000x2000): 1.65x
--------------------------------------------------

Benchmarking 4000x4000 Matrix Multiplication:
CPU Time (4000x4000):